## Importing libraries

In [1]:
import numpy as np
import pandas as pd

## Importing dataset

In [2]:
dataset = pd.read_csv('../../data/data-6-model.csv')
X = dataset.iloc[:, :-2].values
y = dataset.iloc[:, -2].values
print(X[0])
print(y[0])
dataset.head()

[6 'Mathematics' 'Time' 'Games' 49]
Kinesthetic


,grade,subject,lesson,class_interests,lesson_average_mark,teaching_aid_category,teaching_aid
0,6,Mathematics,Time,Games,49,Kinesthetic,Time Difference Board Game
1,6,Mathematics,Time,Videos,69,Audio Visual,Time Theory Exposition
2,6,Mathematics,Circles,Science,68,Visual,DIY Compass
3,6,Mathematics,Time,Videos,60,Audio Visual,Time Concepts Animation
4,6,Mathematics,Subtraction,Art,63,Visual,Posters


## Encode Independent variables

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(sparse_output = False), [1, 2, 3])], remainder='passthrough')
X = ct.fit_transform(X)
print(X[0])

[1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 6 49]


## Encode Dependant variables

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
print(y[0])

3


## Split dataset into Test set and Train set

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

## Feature scaling

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:, 17:] = sc.fit_transform(X_train[:, 17:])
X_test[:, 17:] = sc.transform(X_test[:, 17:])
print(X_train[0])

[1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 6
 -0.8109449345880856]


## Train KNN model on the Train set

In [7]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)

KNeighborsClassifier()

## Predict the Test set results

In [8]:
y_pred = classifier.predict(X_test)

## Create Confusion Matrix

In [9]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))

[[ 3  0  0  0  0]
 [ 0 26  0  0  0]
 [ 0  0  5  0  0]
 [ 0  2  1 39  0]
 [ 0  1  0  3 46]]
0.9444444444444444


## Applying K-Fold Cross Validation

In [10]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy : {:.2f}".format(accuracies.mean()*100))
print("Standard Deviation : {:.2f}".format(accuracies.std()*100))

Accuracy : 90.82
Standard Deviation : 2.71


## Applying GridSearch to find the best hyperparameters

In [11]:
from sklearn.model_selection import GridSearchCV
parameters = [{
    'n_neighbors': range(1, 30, 2),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
    'leaf_size': range(1, 50, 5),
    }]
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           cv = 10,
                           n_jobs = -1)
grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy : {:.2f}".format(best_accuracy*100))
print("Best parameters : ", best_parameters)

Best Accuracy : 94.40
Best parameters :  {'leaf_size': 1, 'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'uniform'}
